In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, roc_curve, auc, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [49]:
df = pd.read_csv('./customer_segmentation_train.csv')

fill_na_mappings = {
    'Work_Experience': 0,  # Fill with zeros
    'Family_Size': 0,  # Fill with zeros
    'Var_1': "Unknown",  # Fill with a constant value
    'Profession': 'Unknown',  # Fill with a constant value
    'Graduated': 'Unknown',
    'Ever_Married': 'Unknown' # Fill with another constant value
}

for col, fill_value in fill_na_mappings.items():
    df[col] = df[col].fillna(fill_value)
    
label_encodings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Spending_Score': {'Low': 0, 'Average': 1, 'High': 2}
}

for col, mapping in label_encodings.items():
    df[col] = df[col].map(mapping)
    df[col].astype(int)
    
X = df.drop(columns=["Segmentation"])
Y = df["Segmentation"]

display(df.isna().sum())

X = X.drop(columns=["ID"])

numeric_cols = X.select_dtypes(include=["int", "float"]).columns.tolist()
# categorical_contionious_cols = ['Gender', 'Spending_Score']
categorical_one_hot_cols = ["Profession", "Var_1", "Ever_Married", 'Graduated']

train_data, val_data, train_y, val_y = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)


numeric_transformer = Pipeline(steps=[
    ('scaler', 
     MinMaxScaler()
     )
])

categorical_one_hot_transformer = Pipeline(steps=[
    ('onehot', 
     OneHotEncoder(handle_unknown='ignore')
     )
])

# Комбінуємо трансформери для різних типів колонок в один препроцесор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat_cont', categorical_one_hot_transformer, categorical_one_hot_cols),
    ])

display(train_data.isna().sum())


train_preprocessed = preprocessor.fit_transform(train_data)
val_preprocessed = preprocessor.transform(val_data)

not_cat_names = train_data.select_dtypes(exclude=["object"])
new_columns = list(not_cat_names.columns) + list(preprocessor["cat_cont"].get_feature_names_out(categorical_one_hot_cols))

print(len(new_columns))
print(train_preprocessed.shape)

preprocessed_train = pd.DataFrame(train_preprocessed, columns=new_columns)
preprocessed_val = pd.DataFrame(val_preprocessed, columns=new_columns)


preprocessed_train
preprocessed_val

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

29
(6454, 29)


,Gender,Age,Work_Experience,Spending_Score,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,...,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Var_1_Unknown,Ever_Married_No,Ever_Married_Unknown,Ever_Married_Yes,Graduated_No,Graduated_Unknown,Graduated_Yes
0,0.0,0.323944,0.857143,0.5,0.333333,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.042254,0.285714,0.0,0.444444,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.253521,0.000000,0.0,0.111111,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.126761,0.000000,0.0,0.222222,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.000000,0.000000,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,1.0,0.211268,0.500000,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1610,0.0,0.802817,0.071429,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1611,1.0,0.436620,0.214286,0.0,0.222222,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1612,0.0,0.352113,0.428571,0.0,0.222222,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках і порівняти лін регресію на апсампл даних без категоріальних ознак.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [59]:
from imblearn.over_sampling import SMOTE, SMOTENC

smote_resampler = SMOTE(random_state=42)
smotenc_resampler = SMOTENC(random_state=42, categorical_features=list(preprocessor["cat_cont"].get_feature_names_out(categorical_one_hot_cols)))

x_test_resampled_smote, y_test_resampled_smote = smote_resampler.fit_resample(preprocessed_train[list(not_cat_names.columns)], train_y)

x_test_resampled_smotenc, y_test_resampled_smotenc = smotenc_resampler.fit_resample(preprocessed_train, train_y)

display(x_test_resampled_smote)
display(x_test_resampled_smotenc)


,Gender,Age,Work_Experience,Spending_Score,Family_Size
0,1.0,0.197183,0.642857,0.0,0.111111
1,0.0,0.760563,0.000000,0.5,0.222222
2,1.0,0.211268,0.071429,0.0,0.444444
3,1.0,0.422535,0.000000,0.5,0.666667
4,1.0,0.140845,0.642857,0.0,0.111111
...,...,...,...,...,...
7251,1.0,0.718310,0.000000,1.0,0.222222
7252,0.0,0.152804,0.316038,0.0,0.380503
7253,0.0,0.402984,0.071429,0.5,0.444444
7254,0.0,0.323944,0.071429,0.5,0.222222


,Gender,Age,Work_Experience,Spending_Score,Family_Size,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,...,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Var_1_Unknown,Ever_Married_No,Ever_Married_Unknown,Ever_Married_Yes,Graduated_No,Graduated_Unknown,Graduated_Yes
0,1.0,0.197183,0.642857,0.0,0.111111,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.760563,0.000000,0.5,0.222222,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,0.211268,0.071429,0.0,0.444444,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.422535,0.000000,0.5,0.666667,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.140845,0.642857,0.0,0.111111,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7251,1.0,0.758038,0.000000,1.0,0.222222,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7252,0.0,0.241962,0.480459,0.0,0.380503,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7253,0.0,0.457655,0.071429,0.5,0.401314,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7254,0.0,0.323944,0.071429,0.5,0.222222,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [79]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf_smote = OneVsRestClassifier(LogisticRegression())
clf_smotenc = OneVsRestClassifier(LogisticRegression())

clf_smote.fit(x_test_resampled_smote, y_test_resampled_smote)
clf_smotenc.fit(x_test_resampled_smotenc, y_test_resampled_smotenc)


smote_prediction = clf_smote.predict(preprocessed_val[list(not_cat_names.columns)])
smotenc_prediction = clf_smotenc.predict(preprocessed_val)

display(val_y.map({"A": 0, "B": 1, "C": 2, "D": 3}))
display(smote_prediction)

smote_report = classification_report(val_y, smote_prediction)
smotenc_report = classification_report(val_y, smotenc_prediction)

print("smote report:")
print(smote_report)
print("smotenc report:")
print(smotenc_report)


6862    0
3396    3
6302    1
3329    3
7901    3
       ..
6635    2
5128    0
2289    1
5285    1
514     2
Name: Segmentation, Length: 1614, dtype: int64

array(['A', 'D', 'D', ..., 'A', 'A', 'C'], dtype='<U1')

smote report:
              precision    recall  f1-score   support

           A       0.37      0.35      0.36       394
           B       0.29      0.08      0.12       372
           C       0.39      0.49      0.43       394
           D       0.53      0.75      0.62       454

    accuracy                           0.43      1614
   macro avg       0.40      0.42      0.38      1614
weighted avg       0.40      0.43      0.40      1614

smotenc report:
              precision    recall  f1-score   support

           A       0.41      0.47      0.44       394
           B       0.41      0.21      0.28       372
           C       0.51      0.63      0.56       394
           D       0.66      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.49      1614
weighted avg       0.51      0.52      0.50      1614



We can for sure say that SMOTENC performed much better than SMOTE. However the performance is still not so good.